# cpu price prediction app

In [1]:
import numpy as np 
import pandas as pd

In [2]:
data = pd.read_csv(r"C:\Users\ma516\OneDrive\Desktop\Machine Learning Projects\CPU Performance Dataset\CPU_benchmark_v4.csv")
data.head()

,cpuName,price,cpuMark,cpuValue,threadMark,threadValue,TDP,powerPerf,cores,testDate,socket,category
0,AMD Ryzen Threadripper PRO 5995WX,NaN,108822,NaN,3330,NaN,280.0,388.65,64,2022,sWRX8,Desktop
1,AMD EPYC 7763,7299.99,88338,12.10,2635,0.36,280.0,315.49,64,2021,SP3,Server
2,AMD EPYC 7J13,NaN,86006,NaN,2387,NaN,NaN,NaN,64,2021,unknown,Server
3,AMD EPYC 7713,7060.00,85861,12.16,2727,0.39,225.0,381.6,64,2021,SP3,Server
4,AMD Ryzen Threadripper PRO 3995WX,6807.98,83971,12.33,2626,0.39,280.0,299.9,64,2020,sWRX8,Desktop


# Feature Extraction

In [3]:
# Check all the missing value
data.isnull().sum()

cpuName           0
price          1858
cpuMark           0
cpuValue       1858
threadMark        0
threadValue    1858
TDP             685
powerPerf       685
cores             0
testDate          0
socket            0
category          0
dtype: int64

In [50]:
# Fil The all Missing Value
data['price'] = data['price'].fillna(data['price'].mean())
data['cpuValue'] = data['cpuValue'].fillna(data['cpuValue'].mean())
data['threadValue'] = data['threadValue'].fillna(data['threadValue'].mean())
data['powerPerf'] = data['powerPerf'].fillna(method = 'ffill')
data['TDP'] = data['TDP'].fillna(method = 'ffill')
data["powerPerf"] = data["powerPerf"].fillna(1)


C:\Users\ma516\AppData\Local\Temp\ipykernel_7156\2352841256.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['powerPerf'] = data['powerPerf'].fillna(method = 'ffill')
C:\Users\ma516\AppData\Local\Temp\ipykernel_7156\2352841256.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['TDP'] = data['TDP'].fillna(method = 'ffill')


In [52]:
data.isnull().sum()

cpuName        0
price          0
cpuMark        0
cpuValue       0
threadMark     0
threadValue    0
TDP            0
powerPerf      0
cores          0
testDate       0
socket         0
category       0
dtype: int64

In [53]:
# known total no of rows and columns
data.shape

(3825, 12)

In [54]:
# known about my dataset
data.describe()

,price,cpuMark,cpuValue,threadMark,threadValue,cores,testDate
count,3825.000000,3825.000000,3825.000000,3825.000000,3825.000000,3825.000000,3825.000000
mean,441.500946,5992.305882,35.354911,1391.107451,15.189009,5.052288,2015.027974
std,650.647576,9617.551479,25.678033,815.667528,12.839840,6.185833,4.597616
min,3.990000,77.000000,0.220000,75.000000,0.130000,1.000000,2007.000000
25%,150.110000,943.000000,24.720000,729.000000,9.430000,2.000000,2011.000000
50%,441.500946,2331.000000,35.354911,1274.000000,15.189009,4.000000,2015.000000
75%,441.500946,6643.000000,35.354911,1961.000000,15.189009,6.000000,2020.000000
max,8978.000000,108822.000000,345.330000,4317.000000,267.820000,80.000000,2022.000000


In [55]:
data['category'].unique()

array(['Desktop', 'Server', 'Desktop, Server', 'Unknown', 'Laptop',
       'Laptop, Server', 'Desktop, Laptop', 'Desktop, Mobile/Embedded',
       'Server, Mobile/Embedded', 'Mobile/Embedded',
       'Laptop, Mobile/Embedded', 'Desktop, Laptop, Mobile/Embedded'],
      dtype=object)

In [56]:
data['socket'].unique()

array(['8', '3', '', '4', '4189', '1700', '3647', '2579', '2066', '1744',
       '1200', '2011-3', '7', '-2066', '1787', '6', '1151-2', '-3647',
       '2518', '2011-1', '1440', '2011', '42', '1449', '1667', '1526',
       '1356', '1598', '1528', '1344', '1151', '2227', '2270', '-1526',
       '5', '34', '1150', '1364', '1366', '946947', '17001700', '946',
       '1155', '-1356', '988', '32', '1224', '1377', '1338', '1310',
       '98812', '1140', '1493', '1296', '2', '1090', '-1016', '23',
       '1156', '1023', '1168', '1207', '115511', '9881', '1515', '771',
       '1283', '775', '604', '1', '2938-', '1234', '2940-', '1288',
       '1567', '11', '12', '478', '7693', '3769-', '1170', '1380', '603',
       '479478', '8272', '479', '13', '940', '22', '-1023', '956', '592',
       '14', '939', '812', '-478', '775775', '754', '559', '4131', '400',
       '437', '111', '-4760', '-4794', '-479', '478-4', '370', '441',
       '462', '518', '423'], dtype=object)

In [57]:
# Split dataset into X and Y
x = data.drop(columns=["testDate" ,"price" , 'category' ]  )
y = data['price']

# Label Encoder

In [58]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()

In [59]:
x['cpuName'] = LE.fit_transform(x['cpuName'])
x['socket'] = LE.fit_transform(x['socket'])


In [60]:
cols = [ 'cpuName', 'socket', 'TDP', 'powerPerf']

for col in cols:
    data[col] = (
        data[col]
        .astype(str)  # ensure string
        .str.replace(",", "", regex=False)  # remove commas
        .str.replace(r'[^0-9.\-]', '', regex=True)  # remove other junk like $ or W
    )
    x[col] = pd.to_numeric(data[col], errors='coerce')  # convert to float


In [61]:
x['cpuName'] = x['cpuName'].fillna(x['cpuName'].mean())
x['socket'] = x['socket'].fillna(x['socket'].mean())


In [62]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3825 entries, 0 to 3824
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   cpuName      3825 non-null   float64
 1   cpuMark      3825 non-null   int64  
 2   cpuValue     3825 non-null   float64
 3   threadMark   3825 non-null   int64  
 4   threadValue  3825 non-null   float64
 5   TDP          3825 non-null   float64
 6   powerPerf    3825 non-null   float64
 7   cores        3825 non-null   int64  
 8   socket       3825 non-null   float64
dtypes: float64(6), int64(3)
memory usage: 269.1 KB


In [87]:
x.head(5)

,cpuName,cpuMark,cpuValue,threadMark,threadValue,TDP,powerPerf,cores,socket
0,5995.0,108822,35.354911,3330,15.189009,280.0,388.65,64,8.000000
1,7763.0,88338,12.100000,2635,0.360000,280.0,315.49,64,3.000000
2,713.0,86006,35.354911,2387,15.189009,280.0,315.49,64,15149.013894
3,7713.0,85861,12.160000,2727,0.390000,225.0,381.60,64,3.000000
4,3995.0,83971,12.330000,2626,0.390000,280.0,299.90,64,8.000000


In [88]:
y.head(5)

0     441.500946
1    7299.990000
2     441.500946
3    7060.000000
4    6807.980000
Name: price, dtype: float64

# Train Our Data

In [66]:
from sklearn.model_selection import train_test_split

In [67]:
x_train , x_test , y_train , y_test = train_test_split(x,y,random_state=2,test_size=0.2)

In [68]:
print(x.shape , x_train.shape , y_train.shape)

(3825, 9) (3060, 9) (3060,)


In [69]:
from sklearn.tree import DecisionTreeRegressor

In [90]:
model = DecisionTreeRegressor(max_depth=4)

In [91]:
model.fit(x_train , y_train)

DecisionTreeRegressor(max_depth=4)

# Find the R2_score

In [92]:
from sklearn.metrics import r2_score

In [93]:
trained_data = model.predict(x_train)
r2_score_of_trained_data = r2_score(y_train,trained_data)
print("r2_score_of_trained_data : " ,r2_score_of_trained_data)

r2_score_of_trained_data :  0.9470259177816888


In [94]:
tested_data = model.predict(x_test)
r2_score_of_tested_data = r2_score(y_test,tested_data)
print("r2_score_of_tested_data : " ,r2_score_of_tested_data)

r2_score_of_tested_data :  0.9315167286594709


In [95]:
# input our data
input_data = (3995.0,	83971	,12.330000,	2626	,0.390000,	280.0,	299.90,	64,	8.000000 )
# convert our data into array
arrayed_data = np.asarray(input_data)
# Reshaped our Data
reshaped_data = arrayed_data.reshape(1,-1)
# predict our data
prediction =  model.predict(reshaped_data)
print(prediction)

[8225.89333333]


C:\Users\ma516\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


# make a model file to connect our data

In [96]:
import pickle

In [97]:
pickle.dump(model , open('cpu_price1.sav' , 'wb'))

In [98]:
loaded_model = pickle.load(open('cpu_price1.sav' , 'rb'))

In [99]:
input_data = (3995.0,	83971	,12.330000,	2626	,0.390000,	280.0,	299.90,	64,	8.000000 )
arrayed_data = np.asarray(input_data)
reshaped_data = arrayed_data.reshape(1,-1)
prediction =  model.predict(reshaped_data)
print(prediction)

[8225.89333333]


C:\Users\ma516\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
